<a href="https://colab.research.google.com/github/marfrlv/gpt-propaganda-detection/blob/main/output_evaluation_all_conditions_with_no_prop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries
import pandas as pd
import os
from io import StringIO
import io
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import json

In [ ]:
# Golden Dataset
golden_full_df_path = '/content/drive/MyDrive/thesis/semeval_2020_task_11_corpus/golden_labels_df_full.csv'
golden_full_df = pd.read_csv(golden_full_df_path)

# additional examples df
add_exm_df_path = '/content/drive/MyDrive/thesis/semeval_2020_task_11_corpus/additional_examples_for_chatgpt.csv'

# paths to the folders with articles
train_art_fold = '/content/drive/MyDrive/thesis/semeval_2020_task_11_corpus/train-articles'
dev_art_fold = '/content/drive/MyDrive/thesis/semeval_2020_task_11_corpus/dev-articles'

# article IDs used in the experiment
article_ids_409_path = '/content/drive/MyDrive/thesis/article_IDs_exp_used.json'

# all articles folder path
all_art_fold_path = '/content/drive/MyDrive/thesis/all_articles'

# 1. Preliminary check of raw texts

In [ ]:
# read the csv files
con1_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0.7/con1_df_no_prop_checked_aug6.csv'
con1_df = pd.read_csv(con1_df_path)

con2_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0.7/con2_df_no_prop_checked_aug6.csv'
con2_df = pd.read_csv(con2_df_path)

con3_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0.7/con3_df_no_prop_checked_aug6.csv'
con3_df = pd.read_csv(con3_df_path)

con4_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0.7/con4_df_no_prop_checked_aug6.csv'
con4_df = pd.read_csv(con4_df_path)

con5_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0/con5_df_no_prop_checked_aug6.csv'
con5_df = pd.read_csv(con5_df_path)


con6_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0/con6_df_no_prop_checked_aug6.csv'
con6_df = pd.read_csv(con6_df_path)


con7_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0/con7_df_no_prop_checked_aug6.csv'
con7_df = pd.read_csv(con7_df_path)


con8_df_path = '/content/drive/MyDrive/thesis/finalized_dfs_aug7/temperature_0/con8_df_no_prop_checked_aug6.csv'
con8_df = pd.read_csv(con8_df_path)

dfs = [con1_df, con2_df, con3_df, con4_df,con5_df, con6_df, con7_df, con8_df ] # raw dfs
dfs_names = ['Condition 1', 'Condition 2', 'Condition 3', 'Condition 4',
             'Condition 5', 'Condition 6', 'Condition 7', 'Condition 8']

In [ ]:
# function to check if text spans exist in corresponding articles
def pre_check_text_spans(df, folder_path):
    results = []
    for index, row in df.iterrows():
        article_id = row['Article ID']
        text_span = row['Text Span']

        # convert text span to string if it's not already
        if not isinstance(text_span, str):
            text_span = str(text_span)

        # construct filename based on article ID
        filename = os.path.join(folder_path, f'{article_id}.txt')

        # check if file exists
        if os.path.isfile(filename):
            # read the text content of the file
            with open(filename, 'r', encoding='utf-8') as file:
                article_text = file.read()

                # check if article_text is not NaN and text span exists in the article text
                if isinstance(article_text, str) and text_span in article_text:
                    results.append(True)
                else:
                    results.append(False)
        else:
            results.append(False)

    # create a new column 'Text Span Presence' with the results
    df = df.copy()  # create a copy to avoid the SettingWithCopyWarning
    df['Text Span Presence'] = results

    return df

# function to count values and print the counts and their percentages
def count_values(df, column):
    value_counts = df[column].value_counts()
    value_counts_percent = df[column].value_counts(normalize=True) * 100
    value_counts_percent = value_counts_percent.round(2)

    print(value_counts)
    print()
    print(value_counts_percent)

folder_path = all_art_fold_path

In [ ]:
# apply the functions and print the outputs
dfs_checked = []
for df, df_name in zip(dfs, dfs_names):
    print(df_name)
    df = pre_check_text_spans(df, folder_path)
    dfs_checked.append(df)
    count_values(df, 'Text Span Presence')
    print("_________________")
    print()
    print()
    print()

Condition 1
Text Span Presence
True     3602
False    2276
Name: count, dtype: int64

Text Span Presence
True     61.28
False    38.72
Name: proportion, dtype: float64
_________________



Condition 2
Text Span Presence
True     3280
False    2455
Name: count, dtype: int64

Text Span Presence
True     57.19
False    42.81
Name: proportion, dtype: float64
_________________



Condition 3
Text Span Presence
False    2424
True     1545
Name: count, dtype: int64

Text Span Presence
False    61.07
True     38.93
Name: proportion, dtype: float64
_________________



Condition 4
Text Span Presence
False    2088
True     1662
Name: count, dtype: int64

Text Span Presence
False    55.68
True     44.32
Name: proportion, dtype: float64
_________________



Condition 5
Text Span Presence
True     4440
False    2581
Name: count, dtype: int64

Text Span Presence
True     63.24
False    36.76
Name: proportion, dtype: float64
_________________



Condition 6
Text Span Presence
True     4866
False    2

In [ ]:
# correct the 'Text Span Presence" for 'no_propaganda_detected' values
for df in dfs_checked:
  df.loc[df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'

<ipython-input-36-b6d21c20abab>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'no_propaganda_detected' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'


In [ ]:
for df_checked, df_name in zip(dfs_checked, dfs_names):
    print(df_name)
    count_values(df_checked, 'Text Span Presence')
    print("_________________")
    print()
    print()
    print()

Condition 1
Text Span Presence
True                      3602
False                     2268
no_propaganda_detected       8
Name: count, dtype: int64

Text Span Presence
True                      61.28
False                     38.58
no_propaganda_detected     0.14
Name: proportion, dtype: float64
_________________



Condition 2
Text Span Presence
True                      3280
False                     2432
no_propaganda_detected      23
Name: count, dtype: int64

Text Span Presence
True                      57.19
False                     42.41
no_propaganda_detected     0.40
Name: proportion, dtype: float64
_________________



Condition 3
Text Span Presence
False                     2417
True                      1545
no_propaganda_detected       7
Name: count, dtype: int64

Text Span Presence
False                     60.90
True                      38.93
no_propaganda_detected     0.18
Name: proportion, dtype: float64
_________________



Condition 4
Text Span Presence
False    

# 2. Clean, Preprocess, then Check

# Temperature = 0.7

In [ ]:
# version with lemmatization
# download the necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Preprocesses the text by removing all punctuation, converting it to lowercase, removing stop words, and lemmatizing."""
    if not isinstance(text, str):
        return ""

    # remove the strings with references to religious texts
    pattern = r'\d*\s*[A-Z][a-z]+\s+\d+:\d+|\d+:\d+'
    text = re.sub(pattern, '', text)

    # remove all punctuation
    translator = str.maketrans('', '', string.punctuation + '“”’‘()')  # additional punctuation marks
    text = text.translate(translator)

    # tokenize the text
    words = word_tokenize(text)

    # remove stop words and lemmatize
    filtered_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]

    # join the words back into a single string
    text = ' '.join(filtered_text)

    return text

def check_text_spans(df, folder_path):
    """Function to check if each text span from the third column is presented in a corresponding article."""

    def check_span_in_file(text_span, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                return text_span in file_content
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return False

    presence_list = []
    article_ids = []
    techniques = []
    text_spans = []

    for index, row in df.iterrows():
        article_id = row['Article ID']
        technique = row['Propaganda Technique']
        text_span = row['Text Span']

        if not isinstance(article_id, str):
            article_id = str(article_id)
        if not isinstance(text_span, str):
            text_span = ""

        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text_span)
        file_path = os.path.join(folder_path, f"{article_id}.txt")

        if len(sentences) > 1:
            for sentence in sentences:
                if os.path.exists(file_path):
                    presence = check_span_in_file(sentence, file_path)
                    presence_list.append(presence)
                else:
                    presence_list.append(False)  # file does not exist
                article_ids.append(article_id)
                text_spans.append(sentence)
                techniques.append(technique)
        else:
            if os.path.exists(file_path):
                presence = check_span_in_file(text_span, file_path)
                presence_list.append(presence)
            else:
                presence_list.append(False)  # file does not exist
            article_ids.append(article_id)
            techniques.append(technique)
            text_spans.append(text_span)

    new_df = pd.DataFrame({'Article ID': article_ids,
                           'Propaganda Technique': techniques,
                           'Text Span': text_spans,
                           'Text Span Presence': presence_list})

    new_df_false = new_df[new_df['Text Span Presence'] == False]
    new_df_false = new_df_false.apply(lambda row: check_text_span_presence(row, folder_path), axis=1)
    new_df.update(new_df_false)

    return new_df

def check_text_span_presence(df_row, folder_path):
    """Checks whether the preprocessed 'Text Span' from the DataFrame row is present in the corresponding preprocessed text file.
    If present, updates the 'Text Span Presence' to True."""

    article_id = str(df_row['Article ID'])
    text_span = preprocess_text(df_row['Text Span'])

    file_path = os.path.join(folder_path, f"{article_id}.txt")

    try:
        with open(file_path, "r", encoding="utf-8") as file:
            article_content = file.read()
            preprocessed_content = preprocess_text(article_content)
            if text_span in preprocessed_content:
                df_row['Text Span Presence'] = True
    except FileNotFoundError:
        print(f"File {article_id}.txt not found in {folder_path}.")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

    return df_row

def count_values(df, column):
  value_counts = df[column].value_counts()
  value_counts_percent = df['Text Span Presence'].value_counts(normalize=True) * 100
  value_counts_percent = value_counts_percent.round(1)

  print(value_counts)
  print()
  print(value_counts_percent)

def save_df_as_csv(df, folder_path, file_name):
  os.makedirs(folder_path, exist_ok=True)
  file_path = os.path.join(folder_path, file_name)
  df.to_csv(file_path, index=False)

  print(f'The df is saved to {file_path}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# CONDITION 1
final_con1_df = check_text_spans(con1_df, all_art_fold_path)
final_con1_df.loc[final_con1_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con1_df

,Article ID,Propaganda Technique,Text Span,Text Span Presence
0,701225819,"name_calling,labeling",white supremacist icon,True
1,701225819,"name_calling,labeling",hatemonger,True
2,701225819,loaded_language,rightfully so,True
3,701225819,loaded_language,bigoted work,True
4,701225819,doubt,This was not the first time Zakkout has promot...,True
...,...,...,...,...
6101,763280007,flag-waving,We will not be silenced.,True
6102,763280007,flag-waving,We will not be intimidated,True
6103,763280007,"name_calling,labeling",the idols of “diversity” and “multiculturalism”,True
6104,763280007,thought-terminating_cliches,enough is enough,True


In [ ]:
count_values(final_con1_df, 'Text Span Presence')

Text Span Presence
True                      5781
False                      317
no_propaganda_detected       8
Name: count, dtype: int64

Text Span Presence
True                      94.7
False                      5.2
no_propaganda_detected     0.1
Name: proportion, dtype: float64


In [ ]:
final_con1_df.to_csv('con1_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 2
final_con2_df = check_text_spans(con2_df, all_art_fold_path)
final_con2_df.loc[final_con2_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con2_df

,Article ID,Propaganda Technique,Text Span,Text Span Presence
0,701225819,loaded_language,white supremacist icon,True
1,701225819,loaded_language,hatemonger,True
2,701225819,loaded_language,David Duke’s bigoted work,False
3,701225819,repetition,David Duke,True
4,701225819,loaded_language,venomous,True
...,...,...,...,...
5985,832910505,loaded_language,Never underestimate the power of the swamp's s...,True
5986,832910505,"exaggeration,minimisation",Much of this is about self-defense.,True
5987,832910505,loaded_language,But plenty of it is about malice and spite.,True
5988,832910505,loaded_language,Culture wars are built on tribal rage.,True


In [ ]:
count_values(final_con2_df, 'Text Span Presence')

Text Span Presence
True                      5672
False                      295
no_propaganda_detected      23
Name: count, dtype: int64

Text Span Presence
True                      94.7
False                      4.9
no_propaganda_detected     0.4
Name: proportion, dtype: float64


In [ ]:
final_con2_df.to_csv('con2_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 3
# Download the necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Define stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """Preprocesses the text by removing all punctuation, converting it to lowercase, removing stop words, and lemmatizing."""
    if not isinstance(text, str):
        return ""

    # Remove the strings with references to religious texts
    pattern = r'\d*\s*[A-Z][a-z]+\s+\d+:\d+|\d+:\d+'
    text = re.sub(pattern, '', text)

    # Remove all punctuation
    translator = str.maketrans('', '', string.punctuation + '“”’‘()')  # Additional punctuation marks
    text = text.translate(translator)

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stop words and lemmatize
    filtered_text = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]

    # Join the words back into a single string
    text = ' '.join(filtered_text)

    return text

def check_text_spans(df, folder_path):
    """Function to check if each text span from the third column is presented in a corresponding article."""

    def check_span_in_file(text_span, file_path):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()
                return text_span in file_content
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return False

    presence_list = []
    article_ids = []
    techniques = []
    text_spans = []
    explanations = []

    for index, row in df.iterrows():
        article_id = row['Article ID']
        technique = row['Propaganda Technique']
        text_span = row['Text Span']
        explanation = row['Explanation']

        if not isinstance(article_id, str):
            article_id = str(article_id)
        if not isinstance(text_span, str):
            text_span = ""

        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text_span)
        file_path = os.path.join(folder_path, f"{article_id}.txt")

        if len(sentences) > 1:
            for sentence in sentences:
                if os.path.exists(file_path):
                    presence = check_span_in_file(sentence, file_path)
                    presence_list.append(presence)
                else:
                    presence_list.append(False)  # File does not exist
                article_ids.append(article_id)
                techniques.append(technique)
                text_spans.append(sentence)
                explanations.append(explanation)
        else:
            if os.path.exists(file_path):
                presence = check_span_in_file(text_span, file_path)
                presence_list.append(presence)
            else:
                presence_list.append(False)  # File does not exist
            article_ids.append(article_id)
            techniques.append(technique)
            text_spans.append(text_span)
            explanations.append(explanation)

    new_df = pd.DataFrame({'Article ID': article_ids,
                           'Propaganda Technique': techniques,
                           'Text Span': text_spans,
                           'Explanation': explanations,
                           'Text Span Presence': presence_list})

    new_df_false = new_df[new_df['Text Span Presence'] == False]
    new_df_false = new_df_false.apply(lambda row: check_text_span_presence(row, folder_path), axis=1)
    new_df.update(new_df_false)

    return new_df

def check_text_span_presence(df_row, folder_path):
    """Checks whether the preprocessed 'Text Span' from the DataFrame row is present in the corresponding preprocessed text file.
    If present, updates the 'Text Span Presence' to True."""

    article_id = str(df_row['Article ID'])
    text_span = preprocess_text(df_row['Text Span'])

    file_path = os.path.join(folder_path, f"{article_id}.txt")

    try:
        with open(file_path, "r", encoding="utf-8") as file:
            article_content = file.read()
            preprocessed_content = preprocess_text(article_content)
            if text_span in preprocessed_content:
                df_row['Text Span Presence'] = True
    except FileNotFoundError:
        print(f"File {article_id}.txt not found in {folder_path}.")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

    return df_row

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# CONDITION 3
final_con3_df = check_text_spans(con3_df, all_art_fold_path)
final_con3_df.loc[final_con3_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con3_df

,Article ID,Propaganda Technique,Text Span,Explanation,Text Span Presence
0,788056108,loaded_language,"""cunning response""",This phrase carries a strong negative emotiona...,True
1,788056108,loaded_language,"""monster once known as Cardinal McCarrick""","The word ""monster"" is emotionally charged to p...",True
2,788056108,"exaggeration,minimisation","""power-mad ecclesiastical tyrant""",The phrase exaggerates Bergoglio's characteris...,True
3,788056108,"name_calling,labeling","""preening, Internet-created neo-Catholic doyenne""",This label is derogatory and aims to diminish ...,True
4,788056108,doubt,"""Whether Francis knows it or not, this is how ...",This introduces uncertainty about Bergoglio's ...,True
...,...,...,...,...,...
4189,763280007,"bandwagon,reductio_ad_hitlerum","""And I tell you, to the governments.",This statement suggests that opposing the gove...,True
4190,763280007,"bandwagon,reductio_ad_hitlerum","You can throw us in jail, but you will never d...",This statement suggests that opposing the gove...,True
4191,763280007,"bandwagon,reductio_ad_hitlerum","Because, my friends, for every Tommy whom you ...",This statement suggests that opposing the gove...,True
4192,763280007,causal_oversimplification,"""If there’s homosexual men, the punishment is ...",This simplifies the complex issues surrounding...,True


In [ ]:
count_values(final_con3_df, 'Text Span Presence')

Text Span Presence
True                      3846
False                      341
no_propaganda_detected       7
Name: count, dtype: int64

Text Span Presence
True                      91.7
False                      8.1
no_propaganda_detected     0.2
Name: proportion, dtype: float64


In [ ]:
final_con3_df.to_csv('con3_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 4
final_con4_df = check_text_spans(con4_df, all_art_fold_path)
final_con4_df.loc[final_con4_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con4_df

,Article ID,Propaganda Technique,Text Span,Explanation,Text Span Presence
0,701225819,loaded_language,"white supremacist icon, hatemonger, bigoted wo...",These terms carry a strong emotional impact to...,False
1,701225819,"name_calling,labeling","hatemonger, bigoted work, venomous, grotesque ...",These labels create a negative image of the pe...,False
2,701225819,repetition,"David Duke videos, David Duke, David Duke’s bi...",The constant repetition of David Duke’s associ...,False
3,701225819,"exaggeration,minimisation",the biggest lie ever foisted upon humanity,This phrase exaggerates the claim about the Ho...,True
4,701225819,appeal_to_fear-prejudice,"the destruction of European Mankind..., a Jew ...",These statements incite fear and prejudice by ...,False
...,...,...,...,...,...
3966,763412406,doubt,"""Why is it so easy for judges to make rulings ...",Implies doubt and suspicion towards the judici...,True
3967,763412406,black-and-white_fallacy,"""You either submit, or you resist.""",Presents only two opposing actions as possible...,True
3968,763412406,"whataboutism,straw_men,red_herring","""Why is it so easy to jail Tommy Robinson, whe...",This comparison distracts from the legal speci...,False
3969,763412406,"bandwagon,reductio_ad_hitlerum","""Fantastic crowd!""",Implies that the high turnout and enthusiasm o...,True


In [ ]:
count_values(final_con4_df, 'Text Span Presence')

Text Span Presence
True                      3708
False                      249
no_propaganda_detected      14
Name: count, dtype: int64

Text Span Presence
True                      93.4
False                      6.3
no_propaganda_detected     0.4
Name: proportion, dtype: float64


In [ ]:
final_con4_df.to_csv('con4_final_df_aug24.csv', index=False)

# Temperature = 0

In [ ]:
# CONDITION 5
final_con5_df = check_text_spans(con5_df, all_art_fold_path)
final_con5_df.loc[final_con5_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con5_df

,Article ID,Propaganda Technique,Text Span,Text Span Presence
0,701225819,loaded_language,white supremacist icon,True
1,701225819,loaded_language,hatemonger,True
2,701225819,loaded_language,bigoted work,True
3,701225819,loaded_language,documented bigotry,True
4,701225819,loaded_language,Jewish Zionist Agent,True
...,...,...,...,...
7223,773937361,causal_oversimplification,"""The scandal of McCarrick is simply a toxic em...",True
7224,773937361,flag-waving,"""Our goal must be preserving the precious trea...",True
7225,773937361,appeal_to_authority,"""Rod Dreher[1] is doing excellent work compili...",True
7226,773937361,"bandwagon,reductio_ad_hitlerum","""Whatever you might think of Archbishop Lefebv...",True


In [ ]:
count_values(final_con5_df, 'Text Span Presence')

Text Span Presence
True                      6890
False                      321
no_propaganda_detected      17
Name: count, dtype: int64

Text Span Presence
True                      95.3
False                      4.4
no_propaganda_detected     0.2
Name: proportion, dtype: float64


In [ ]:
final_con5_df.to_csv('con5_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 6
final_con6_df = check_text_spans(con6_df, all_art_fold_path)
final_con6_df.loc[final_con6_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con6_df

,Article ID,Propaganda Technique,Text Span,Text Span Presence
0,701225819,loaded_language,white supremacist icon,True
1,701225819,loaded_language,hatemonger,True
2,701225819,loaded_language,bigoted work,True
3,701225819,loaded_language,venomous,True
4,701225819,loaded_language,promotes bigotry,False
...,...,...,...,...
7702,789615291,doubt,"And don’t forget, the old sorta, kinda moderat...",True
7703,789615291,"whataboutism,straw_men,red_herring",doesn’t Keith Ellison’s accuser deserve to be ...,True
7704,789615291,black-and-white_fallacy,"every accusation of sexual assault, no matter ...",True
7705,789615291,"bandwagon,reductio_ad_hitlerum",the eleven male Republican Senators on the Jud...,True


In [ ]:
count_values(final_con6_df, 'Text Span Presence')

Text Span Presence
True                      7287
False                      398
no_propaganda_detected      22
Name: count, dtype: int64

Text Span Presence
True                      94.6
False                      5.2
no_propaganda_detected     0.3
Name: proportion, dtype: float64


In [ ]:
final_con6_df.to_csv('con6_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 7
final_con7_df = check_text_spans(con7_df, all_art_fold_path)
final_con7_df.loc[final_con7_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con7_df

,Article ID,Propaganda Technique,Text Span,Explanation,Text Span Presence
0,701225819,loaded_language,hatemonger,"The term ""hatemonger"" carries a strong negativ...",True
1,701225819,loaded_language,bigoted work,"The phrase ""bigoted work"" is used to evoke a n...",True
2,701225819,loaded_language,venomous,"Describing the video as ""venomous"" uses emotio...",True
3,701225819,loaded_language,grotesque cartoon,"The adjective ""grotesque"" is used to provoke d...",True
4,701225819,loaded_language,absurd notion,"The phrase ""absurd notion"" is used to dismiss ...",True
...,...,...,...,...,...
4318,763280007,doubt,"""the beginning of an authoritarian police state""",This phrase casts doubt on the integrity and i...,True
4319,763280007,appeal_to_fear-prejudice,"""Muslim rape gangs""",The repeated use of this phrase is intended to...,True
4320,763280007,"bandwagon,reductio_ad_hitlerum","""we, my friends, we are the people.""",This statement implies that the speaker's grou...,True
4321,763280007,causal_oversimplification,"""because he violated a court order demanding t...",This simplifies the complex legal and social i...,True


In [ ]:
count_values(final_con7_df, 'Text Span Presence')

Text Span Presence
True                      4110
False                      201
no_propaganda_detected      12
Name: count, dtype: int64

Text Span Presence
True                      95.1
False                      4.6
no_propaganda_detected     0.3
Name: proportion, dtype: float64


In [ ]:
final_con7_df.to_csv('con7_final_df_aug24.csv', index=False)

In [ ]:
# CONDITION 8
final_con8_df = check_text_spans(con8_df, all_art_fold_path)
final_con8_df.loc[final_con8_df['Propaganda Technique'] == 'no_propaganda_detected', 'Text Span Presence'] = 'no_propaganda_detected'
final_con8_df

,Article ID,Propaganda Technique,Text Span,Explanation,Text Span Presence
0,701225819,loaded_language,"""white supremacist icon"", ""hatemonger"", ""bigot...","These phrases carry strong emotional impact, a...",False
1,701225819,"name_calling,labeling","""white supremacist icon"", ""hatemonger"", ""Jewis...",These labels are used to invoke hatred or disd...,False
2,701225819,repetition,"""David Duke"", ""bigoted"", ""Jewish"", ""hate""",The repeated use of these words throughout the...,False
3,701225819,"exaggeration,minimisation","""the biggest lie ever foisted upon humanity"", ...",These statements exaggerate the significance o...,False
4,701225819,appeal_to_fear-prejudice,"""the destruction of European Mankind"", ""Death ...",These statements instill fear and prejudice ag...,False
...,...,...,...,...,...
4072,111111132,black-and-white_fallacy,"""Even if 99.9 percent of the people in the com...",This text span presents the issue as having on...,True
4073,111111132,"exaggeration,minimisation","""If all of the Israelites had chosen to follow...",This text span from a lesson exaggerates the p...,True
4074,111111132,doubt,"""Students at that age really aren’t prepared t...",This text span casts doubt on the ability of y...,True
4075,111111132,appeal_to_authority,"""Charles C.",This text span cites an authority figure in re...,True


In [ ]:
count_values(final_con8_df, 'Text Span Presence')

Text Span Presence
True                      3975
False                       85
no_propaganda_detected      17
Name: count, dtype: int64

Text Span Presence
True                      97.5
False                      2.1
no_propaganda_detected     0.4
Name: proportion, dtype: float64


In [ ]:
final_con8_df.to_csv('con8_final_df_aug24.csv', index=False)